In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-06 12:12:51--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-06 12:12:51 (32.2 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [1]:
import minsearch

In [2]:
import json

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [7]:
q = 'the course has already started, can I still enroll?'

In [8]:
index.fit(documents)

In [9]:
from openai import OpenAI

In [10]:
# download from https://ollama.com/download
# run `ollama pull llama3`

In [11]:
client = OpenAI(base_url = 'http://localhost:11434/v1', api_key='ollama' )

In [12]:
response = client.chat.completions.create(
    model='llama3',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"It's not uncommon to want to join a course or program after it's already begun. The answer depends on the specific course and the policies of the institution offering it.\n\nHere are some possible scenarios:\n\n1. **Late Enrollment**: Some courses or programs may allow late enrollment, especially if you're still within a reasonable time frame (e.g., during the first few weeks). In this case, you can contact the institution's registrar's office or the course instructor to ask about enrolling late.\n2. **Permission from Instructor**: If the course has already started, you might need permission from the instructor to enroll. This is more likely if the course has a limited number of seats available or if the instructor wants to assess your preparedness for the material. You can reach out to the instructor and explain your situation; they may consider granting permission.\n3. **Add/Drop Period**: Some institutions have an add/drop period during which students can still enroll in courses wi

In [13]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [14]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [15]:
def llm(prompt):
    response = client.chat.completions.create(
        model='llama3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [16]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [17]:
rag(query)

'Based on the provided FAQ database, I will answer your question about running Kafka.\n\nThe question is: how do I run Kafka?\n\nTo run Kafka producer/consumer/KStreams/etc. in the terminal:\n\n1. For Java Kafka:\n    ```\n    java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n    ```\n\n2. For Python Kafka:\n    a. To fix "Module \'kafka\' not found" error, create a virtual environment and run `requirements.txt` and the python files in that environment.\n        ```\n        python -m venv env\n        source env/bin/activate\n        pip install -r ../requirements.txt\n        ```\n\n        or on Windows: \n        ```\n        env\\Scripts\\activate\n        pip install -r ../requirements.txt\n        ```\n\n    b. Run the python file with the virtual environment activated.\n\nIf you encounter a permission denied error when running `build.sh`, run this command in your terminal to fix it:\n```\nchmod +x build.sh\n```'

In [18]:
rag('the course has already started, can I still enroll?')

'Based on the context, since the course has already started, unfortunately, you cannot enroll in the course as enrollment closed before the start date. However, according to the answer provided under "Can I still join the course after the start date?", even if you don\'t register, you\'re still eligible to submit the homeworks.'

In [19]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [20]:
from elasticsearch import Elasticsearch

In [21]:
es_client = Elasticsearch('http://localhost:9200') 

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [29]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [30]:
from tqdm.auto import tqdm

In [31]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [32]:
query = 'I just disovered the course. Can I still join it?'

In [33]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [34]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [35]:
rag(query)

"Based on the provided context, I can answer your question:\n\nYes, you can still join the course after the start date. You're still eligible to submit homeworks, but be aware that there will be deadlines for turning in final projects, so don't leave everything for the last minute."